# 다변량 분석 Logistic Regression

In [1]:
%pip install -r requiremets.txt

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not open requirements file: [Errno 2] No such file or directory: "'requiremets.txt'"


### Evaluation function Define

In [9]:
from typing import Union #힌트 지원에서 Union 사용 가능하게 해준다.
import numpy as np
from sklearn.metrics import confusion_matrix

def perf_eval_clf(y_true: Union[np.array,list], y_pred: Union[np.array,list])->list:
    """
    Input:
    y_true: ndarray of shape (n_samples,)
    y_pred: ndarray of shape (n_samples,)

    Return:
    [TPR,Precision,TNR, Accuracy, BCR, F1]: scores of classification in a list
    """
    tn, fp, fn, tp = confusion_matrix(y_true = y_true, y_pred = y_pred).ravel() #그냥 confusion matrix함수는 matrix를 반환. Ravel로 지표 반환.
    TPR = round(tp/(tp+fn),4) #sencitivity
    TNR = round(tn/(fp+tn),4) #specificity
    PRE = round(tp/(tp+fp),4)
    ACC = round((tp+tn)/(tp+tn+fp+fn),4)
    #BCR = round(np.sqrt(TPR*TNR),4) 내코드임. 왜 아래 처럼 하는지 확인하자.
    #RC = round(tp/(tp+fn),4) Recall인데 TPR과 같다.
    BCR = round(np.sqrt([TPR*TNR])[0],4)
    F1 = round((2*TPR*PRE/(TPR+PRE)),4)

    return [TPR,PRE,TNR, ACC, BCR, F1] 

In [30]:
import pandas as pd

def display_result(result_dict: dict, metric_names: list):

    result_df = pd.DataFrame(result_dict).T #Transpose를 해줘야 index에 dataset name이 들어감.
    result_df.index = list(result_dict.keys())
    result_df.columns = metric_names

    display(result_df)

result_dict = {}



In [12]:
ploan = pd.read_csv("Personal Loan.csv")
ploan.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.scale.html

scale(data, axis, with_mean, with_std) 

- axis : 0이면 각 features 기준이고, 1이면 각 행 기준
- with_mean : True이면 scaling 이전에 centering 작업
- with_std : True이면 데이터를 동일한 분산으로 scaling

In [17]:
from sklearn.preprocessing import scale #standardizer 평균 0 분산을 동일하게 조정해준다.

# Indices for the activated input variables
input_idx = [1,2,3,4,5,6,7,8,10,11,12,13]
target_index = 9 #위 컬럼줌 personal loan 즉 개인 대출 유무를 판별함
ploan_input = ploan.iloc[:, input_idx]
ploan_input_scaled = scale(ploan_input,axis=0,with_mean=True,with_std=True)
ploan_target = ploan.iloc[:, target_index]
ploan_data = pd.concat((ploan_input,ploan_target),axis=1) #personal loan이 가장 끝으로 가게 해줌

ploan_data.head()

,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Securities Account,CD Account,Online,CreditCard,Personal Loan
0,25,1,49,91107,4,1.6,1,0,1,0,0,0,0
1,45,19,34,90089,3,1.5,1,0,1,0,0,0,0
2,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,35,8,45,91330,4,1.0,2,0,0,0,0,1,0


In [18]:
from sklearn.model_selection import train_test_split
seed = 12345
test_size=0.3
ploan_train, ploan_test = train_test_split(ploan_data, test_size = test_size, random_state = seed)

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

- liblinear는 아래 참조

https://m.blog.naver.com/gdpresent/221703566189

In [25]:
#input, output variables in numpy array
#x_trn = ploan_train.drop(columns = ["Personal Loan"]).to_numpy()
#y_trn = ploan_train['Personal Loan'].to_numpy()
x_trn = ploan_train.iloc[:,:-1].to_numpy()
y_trn = ploan_train.iloc[:,-1].to_numpy()

#model setting
from sklearn.linear_model import LogisticRegression
full_lr = LogisticRegression(solver='liblinear',random_state = seed, max_iter = 1e+5)

#Train the model
model = full_lr.fit(x_trn,y_trn)

In [26]:
from scipy.stats import norm
import scipy.stats as stats
def logit_pvalue(model,x,feature_names,cls=0):
    p1 = model.predict_proba(x)
    n1 = len(p1)
    m1 = len(model.coef_[cls]) + 1
    if len(model.intercept_)>1:
        feature_names=['constant'] + feature_names
        coefs = np.concatenate([np.array(model.intercept_[cls].reshape(1,)), model.coef_[cls]])
    elif len(model.intercept_)==1:
        feature_names=['constant'] + feature_names
        coefs = np.concatenate([model.intercept_, model.coef_[0]])
    else:
        coefs = model.coef_[cls]
    x_full = np.matrix(np.insert(np.array(x), 0, 1, axis = 1))
    answ = np.zeros((m1, m1))
    for i in range(n1):
        answ = answ + np.dot(np.transpose(x_full[i, :]), x_full[i, :]) * p1[i,1] * p1[i, 0]
    vcov = np.linalg.inv(np.matrix(answ))
    se = np.sqrt(np.diag(vcov))
    t1 = coefs/se
    p1 = (1 - norm.cdf(abs(t1))) * 2
    pvalue_dict = {key:round(value,4) for key,value in zip(feature_names,p1)}
    display(pd.DataFrame(pvalue_dict,columns=pvalue_dict.keys(),index=['P-value']).T)

feature_names = list(ploan_input.columns)
logit_pvalue(model, x_trn, feature_names=feature_names)

,P-value
constant,1.0000
Age,0.9932
Experience,0.9932
Income,0.0000
ZIP Code,0.2518
Family,0.9966
CCAvg,0.9829
Education,0.9973
Mortgage,0.7006
Securities Account,0.9999


In [37]:
x_test = ploan_test.iloc[:,:-1].to_numpy()
lr_target = y_test = ploan_test.iloc[:,-1].to_numpy()
lr_response = full_lr.predict(x_test)

cm = confusion_matrix(lr_target, lr_response)
cm

array([[642,  20],
       [ 64,  24]], dtype=int64)

In [36]:
clr_results = perf_eval_clf(lr_target,lr_response)
print(clr_results)
result_dict['Personal Loan'] = clr_results
metric_names = ['TPR(Recall)','Precision','TNR','ACC','BCR','F1']
display_result(result_dict=result_dict, metric_names=metric_names)

[0.2727, 0.5455, 0.9698, 0.888, 0.5143, 0.3636]


,TPR(Recall),Precision,TNR,ACC,BCR,F1
Personal Loan,0.2727,0.5455,0.9698,0.888,0.5143,0.3636


# Multi classification
범주의 종류가 3개 이상인 경우 

In [59]:
def perf_eval3(cm: np.array)->list:
    """
    Input:
    Confusion Matrix
    cm: ndarray of shape (n_labels, n_labels) / for 'wine' dataset, n_labels = 3

    Return:
    [ACC, BCR]
    """

    #Acc
    ACC = round(np.sum(np.diag(cm))/np.sum(cm),3) #대각행렬합을 전체 행렬 합으로 나눔

    #각 class에 대한 acc
    A1 = cm[0,0]/np.sum(cm[0,:])
    A2 = cm[1,1]/np.sum(cm[1,:])
    A3 = cm[2,2]/np.sum(cm[2,:])

    BCR = round((A1*A2*A3)**(1/3),3)

    return [ACC,BCR]

result_dict = {}


In [60]:
wine = pd.read_csv("wine.csv")

seed = 12345
test_size = 0.3
wine_trn, wine_tst = train_test_split(wine, test_size=test_size, random_state=seed)

In [61]:
x_trn, y_trn = wine_trn.iloc[:,1:].to_numpy(), wine_trn.iloc[:,0].to_numpy()

ml_logit = LogisticRegression(solver='lbfgs', max_iter = 1e+10)
#multi class 분류이므로 documnet에 따라 lbfgs로 해야함.

ml_logit.fit(x_trn,y_trn)

LogisticRegression(max_iter=10000000000.0)

lbfgs를 사용하면 변수 개수만큼 모델이 나오므로, 각 feature를 따로 확인해야

In [62]:
feature_names = wine.columns.to_list()

feature_coef_dict = {'class 1': {feature:coef for feature, coef in zip(feature_names[1:], ml_logit.coef_[0])},
                     'class 2': {feature:coef for feature, coef in zip(feature_names[1:], ml_logit.coef_[1])},
                     'class 3': {feature:coef for feature, coef in zip(feature_names[1:], ml_logit.coef_[2])}}

pd.DataFrame(feature_coef_dict).T

,Alcohol?2,Malic acid?,Ash?,Alcalinity of ash?,Magnesium?,Total phenols?,Flavanoids?,Nonflavanoid phenols,Proanthocyanins?,Color intensity?,Hue,OD280/OD315 of diluted wines?,Proline?
class 1,0.401847,0.484295,0.605949,-0.203462,-0.022895,0.148597,0.698119,0.106386,0.023003,0.127016,-0.007998,0.488818,0.009698
class 2,-0.622503,-0.716259,-0.660119,0.122128,-0.022191,0.249726,0.297316,-0.042225,0.622515,-0.979478,0.253276,0.105085,-0.008130
class 3,0.220656,0.231964,0.054169,0.081334,0.045086,-0.398323,-0.995436,-0.064162,-0.645519,0.852462,-0.245278,-0.593903,-0.001567


In [63]:
x_tst = wine_tst.iloc[:,1:].to_numpy()
lr_target = y_tst = wine_tst.iloc[:,0].to_numpy()
lr_response = ml_logit.predict(x_tst)
cm = confusion_matrix(lr_target,lr_response);cm

array([[20,  0,  0],
       [ 0, 17,  1],
       [ 0,  0, 16]], dtype=int64)

In [64]:
clf_results = perf_eval3(cm)
result_dict['wine'] = clf_results
metric_names = ['ACC','BCR']
display_result(result_dict,metric_names)

,ACC,BCR
wine,0.981,0.981
